In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist['data'], mnist['target'].astype(int)

# Нормализация
X = X / 255.0

# Разделяем на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [2]:
from sklearn.neural_network import MLPClassifier

# Простая MLP: 2 скрытых слоя по 128 нейронов
mlp = MLPClassifier(hidden_layer_sizes=(128, 128), activation='relu',
                    solver='adam', max_iter=20, random_state=42)

mlp.fit(X_train, y_train)

# Предсказания
y_pred_train = mlp.predict(X_train)
y_pred_test = mlp.predict(X_test)

# Метрики
print("MLP (sklearn) - Train accuracy:", accuracy_score(y_train, y_pred_train))
print("MLP (sklearn) - Test accuracy:", accuracy_score(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))

MLP (sklearn) - Train accuracy: 0.9991964285714285
MLP (sklearn) - Test accuracy: 0.9767142857142858
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      1343
           1       0.99      0.99      0.99      1600
           2       0.98      0.96      0.97      1380
           3       0.96      0.98      0.97      1433
           4       0.98      0.98      0.98      1295
           5       0.98      0.96      0.97      1273
           6       0.98      0.99      0.99      1396
           7       0.99      0.97      0.98      1503
           8       0.97      0.97      0.97      1357
           9       0.97      0.98      0.97      1420

    accuracy                           0.98     14000
   macro avg       0.98      0.98      0.98     14000
weighted avg       0.98      0.98      0.98     14000



E:\Python_LabsAndProjekt\ML\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:785: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.pool  = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1   = nn.Linear(16*4*4, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)
        self.relu  = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 16*4*4)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")


model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

from sklearn.metrics import accuracy_score, classification_report
print("CNN (PyTorch) - Test accuracy:", accuracy_score(y_true, y_pred))
print(classification_report(y_true, y_pred))

100.0%
100.0%
100.0%
100.0%


Epoch 1/5, Loss: 0.2326
Epoch 2/5, Loss: 0.0705
Epoch 3/5, Loss: 0.0500
Epoch 4/5, Loss: 0.0411
Epoch 5/5, Loss: 0.0337
CNN (PyTorch) - Test accuracy: 0.9881
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       980
           1       0.98      1.00      0.99      1135
           2       0.99      0.99      0.99      1032
           3       0.99      0.99      0.99      1010
           4       0.99      0.99      0.99       982
           5       0.98      0.99      0.98       892
           6       0.99      0.99      0.99       958
           7       0.98      0.99      0.98      1028
           8       1.00      0.97      0.98       974
           9       0.99      0.97      0.98      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000



Обе модели показали высокую точность, но CNN продемонстрировала лучшие результаты на тестовой выборке (~98.8% против ~97.7% у MLP) благодаря способности учитывать локальные признаки изображений. Таким образом, сверточная нейронная сеть обеспечивает более точную и устойчивую классификацию по сравнению с обычным MLP.